## Installation

We will need to install a number of libraries to start with.

In [1]:
import networkx as nx
!pip install transformers
!pip install sentencepiece

We use flan alpaca model for speed and local execution.

In [2]:
from transformers import pipeline

prompt = "Write an email about an alpaca that likes flan"
model = pipeline(model="declare-lab/flan-alpaca-xl", device=0)
model(prompt, max_length=128, do_sample=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/50.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.50k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

[{'generated_text': "Dear Alpaca Owner, Our Alpaca name is John – and he is quite a character! He's always up for a snack, so when I hear he's craves flan, I think I might have found a new favorite treat for him. He loves eating crumble, cake, and ice cream – so I thought this is the perfect opportunity for me to share this tasty snack with him. For those of you who want to try this tasty treat, please don't hesitate to let me know! Regards, John"}]

In [3]:
from transformers import pipeline
import networkx as nx
prompt = '''### Instruction:
Who is the first person on the moon?

### Response:'''
def generate(prompt):
  output = model(prompt, do_sample=True, min_length=10, max_length=len(prompt)+128)
  out = output[0]['generated_text']
  if '### Response:' in out:
    out = out.split('### Response:')[1]
  if '### Instruction:' in out:
    out = out.split('### Instruction:')[0]
  return out

## World Description
We describe the world below. We will generate prompts based on this information. The simulation is that of the town of Phandalin, southwest of Neverwinter. This area is chosen because it is easily extendable with multiple regions for a "player" to be able to explore the world once the simulation is done.

In [4]:
world_graph = nx.Graph()

prompt_meta = '''### Instruction:
{}

### Response:'''

town_areas = ["Barthen's Provisions", "Lionshield Coster", "Stonehill Inn", "Phandalin Town Square"]
town_areas = {"Phandalin Town Square": 'Town square of the town of Phandalin.',
              'Stonehill Inn': "In the center of town stands a large, newly built roadhouse of fieldstone and rough-hewn timbers. The common room is filled with locals nursing mugs of ale or cider, all of them eyeing you with curiosity.",
              "Barthen's Provisions": "Barthen’s is the biggest trading post in Phandalin. Its shelves stock most ordinary goods and supplies, including backpacks, bedrolls, rope, and rations. The place is open from sunup to sundown.",
              "Edermath Orchard": "A tidy little cottage beside an apple orchard.",
              "Lionshield Coster": "Hanging above the front door of this modest trading post is a sign shaped like a wooden shield with a blue lion painted on it. This building is owned by the Lionshields, a merchant company based in the city of Yartar, over a hundred miles to the east. They ship finished goods to Phandalin and other small settlements throughout the region, but this outpost has been hard hit by banditry. The most recent Lionshield caravan due in Phandalin never arrived.",
              "Phandalin Miner's Exchange": "The Miner’s Exchange is a trading post where local miners have their valuable finds weighed, measured, and paid out. In the absence of any local lord or authority, the exchange also serves as an unofficial records office, registering claims to various streams and excavations around the area. There isn’t any real gold rush in Phandalin, but enough wealth is hidden in the nearby streams and valleys to support a good number of independent prospectors. The exchange is a great place to meet people who spend a lot of time out and about in the countryside surrounding Phandalin. The guildmaster is an ambitious and calculating human woman named Halia Thornton.",
              "Alderleaf Farm": "A farm owned by the helpful halfling farmer, Qelline Alderleaf.",
              "Shrine of Luck": "Phandalin's only temple is a small shrine made of stones taken from the nearby ruins. It is dedicated to Tymora, goddess of luck and good fortune.",
              "The Sleeping Giant": "This rundown tap house is a dirty, dangerous watering hole at the end of Phandalin’s main street. It is frequented by Redbrand thugs and operated by a surly female dwarf named Grista.",
              "Townmaster’s Hall": "The townmaster’s hall has sturdy stone walls, a pitched wooden roof, and a bell tower at the back. Posted on a board next to the front door is a notice written in Common. It reads: “REWARD — Orcs near Wyvern Tor! Those of a mind to face the orc menace should inquire within.” The notice bears the town’s seal and an indecipherable signature.",
              "Tresendar Manor": "A ruined manor. The Redbrands’ base in Phandalin is a dungeon complex under Tresendar Manor. Before the manor was ruined, its cellars served as safe storage for food and water in the event that the estate was attacked, while an adjoining crypt provided a resting place for the deceased members of the Tresendar family. The Redbrands have since expanded the cellars to suit their own purposes, adding slave pens, workshops, and barracks."
              }
town_people = {"Toblen Stonehill": "Toblen owns a trading post.",
               "Daran Edermath": "Daran is a retired adventurer who lives in a tidy little cottage beside an apple orchard. A fit, silver-haired half-elf well over a hundred years old, Daran is a fighter who served as a marshal and herald for many years in the lands of the Dragon Coast, far to the southeast. Upon retiring, he returned to the Neverwinter region, his original home.",
               "Linene Graywind": "Linene runs a trading post.",
               "Halia Thornton": "Halia is an ambitious and calculating human woman. She is the guildmaster of Phandalin Miner’s Exchange, a trading post where local miners have their valuable finds weighed, measured, and paid out. In her attempts to establish the Miner's Exchange as the closest thing the town has to a governing authority, she acts as more than a simple merchant.",
               "Qelline Alderleaf": "Qelline is a wise female halfling of forty-five, and is a pragmatic farmer who seems to know everything that goes on in town. She is a kind host, and is willing to let the characters stay in her hayloft if they don't want to stay at the Stonehill Inn.",
               "Sister Garaele": "Sister Garaele is an elf cleric of Tymora and a Harper agent.",
               "Harbin Wester": "Harbin is the townmaster of Phandalin. A pompous, old food. Phandalin has no functioning government, but the townsfolk elect someone to serve as townmaster each year. The townmaster serves as a judge in minor disputes and keeps any records that need to be kept.",
               "Terrill Bloodscar": "Terrill is a human ruffian. He wears a grimy scarlet cloak. He is a member of the Redbrand ruffians. He doesn't like adventurers, and wants to rob and kill them.",
               "Conrad Scarface": "Conrad is a human ruffian. He wears a grimy scarlet cloak. He is a member of the Redbrand ruffians. He doesn't like adventurers, and wants to rob and kill them.",
               "Nellie Starsmith": "Nellie is a human ruffian. She wears a grimy scarlet cloak. She is a member of the Redbrand ruffians. She doesn't like adventurers, and wants to rob and kill them.",
               "Valerie Grinblade": "Valerie is a human ruffian. She wears a grimy scarlet cloak. She is a member of the Redbrand ruffians. She doesn't like adventurers, and wants to rob and kill them.",
               }
for town_area in town_areas.keys():
  world_graph.add_node(town_area)
  world_graph.add_edge(town_area, town_area)
for town_area in town_areas.keys():
  world_graph.add_edge(town_area, "Phandalin Town Square")
locations = {}
for i in town_people.keys():
  locations[i] = "Phandalin Town Square"


memories = {}
for i in town_people.keys():
  memories[i] = []
plans = {}
for i in town_people.keys():
  plans[i] = []

global_time = 8
def generate_description_of_area(x):
  text = "It is "+str(global_time)+":00. The location is "+x+"."
  people = []
  for i in locations.keys():
    if locations[i] == x:
      people.append(i)


In [5]:
compressed_memories_all = {}
for name in town_people.keys():
  compressed_memories_all[name] = []

In [6]:
for name in town_people.keys():
  prompt = "You are {}. {} You just woke up in the town of Phandalin and went out to the Town Square. The following people live in the town: {}. What is your goal for today? Be brief, and use at most 20 words and answer from your perspective.".format(name, town_people[name], ', '.join(list(town_people.keys())) )
  plans[name] = generate(prompt_meta.format(prompt))
  print(name, plans[name])

Toblen Stonehill My goal for today is to start a new business in the town.
Daran Edermath I want to explore the town. As a retired adventurer, I'm looking forward to finding interesting places and meeting interesting people. Today I'm setting my sights on the Phandalin Market, which is usually a hub of activity during the day. I'll take any chance I can to talk to people and get to know the town. We may talk politics, find information, or just relax in the sunshine at the end of the day.
Linene Graywind I hope to make some sales from my trading post today.
Halia Thornton My goal for today is to engage in social conversations with the other people in town and grow the Miner's Exchange.
Qelline Alderleaf Qelline's goal for today is to get to know the people in Phandalin.
Sister Garaele Find out information about Harper agents in Phandalin.
Harbin Wester my goal for today is to help the people of Phandalin solve their issues.
Terrill Bloodscar Terrill Bloodscar's goal is to be the leader 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Conrad Scarface Ripping off adventurers. ###Profession: Human ruffian, Redbrand.
Nellie Starsmith I want to rob and murder adventurers in Phandalin.
Valerie Grinblade To target adventurers. 


In [7]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)

  for name in people:
    prompt = "You are {}. {} You are planning to: {}. You are currently in {} with the following description: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, town_people[name], plans[name], location, town_areas[location], str(global_time), ', '.join(people))
    people_description = []
    for i in people:
      people_description.append(i+': '+town_people[i])
    prompt += ' You know the following about people: ' + '. '.join(people_description)
    memory_text = '. '.join(memories[name][-10:])
    prompt += "What do you do in the next hour? Use at most 10 words to explain."
    action_prompts[name] = prompt

In [8]:
action_results = {}
for name in town_people.keys():
  action_results[name] = generate(prompt_meta.format(action_prompts[name]))
  # Now clean the action
  prompt = """
  Convert the following paragraph to first person past tense:
  "{}"
  """.format(action_results[name])
  action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
  print(name, action_results[name])

Token indices sequence length is longer than the specified maximum sequence length for this model (843 > 512). Running this sequence through the model will result in indexing errors


Toblen Stonehill I developed business plans and pitched to investors. I built relationships in town.
Daran Edermath I arrived at the Phandalin Market at 9:00 and talked to people. I made a connection with Toblen Stonehill, Linene Graywind, and Halia Thornton. I was shown their backgrounds and their main concerns. Meanwhile, I was shown the activities of the merchants.
Linene Graywind Linne set up her trading table and interacted with customers and planned for the next hour.
Halia Thornton I reached out to the four people in the square and introduced myself. We had a chat about the town and their goals. I offered to help with the trade and they offered me help to expand my business. Then, we began a formal negotiation for us both to sign. We all went to dinner, allowing me to meet new people, discuss potential solutions and build relationships. At noon, I headed to the city for a meeting with the mayor and other government officials.
Qelline Alderleaf Qelline, Toblen, Daran, Linene, Hal

Collect the memories people observe.

In [9]:
action_prompts = {}
for location in town_areas.keys():
  people = []
  for i in town_people.keys():
    if locations[i] == location:
      people.append(i)

  for name in people:
    for name_two in people:
      memories[name].append('[Time: {}. Person: {}. Memory: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

# Rank Memories

In [10]:
import re
def get_rating(x):
  nums = [int(i) for i in re.findall(r'\d+', x)]
  if len(nums)>0:
    return min(nums)
  else:
    return None

In [11]:
memory_ratings = {}
for name in town_people.keys():
  memory_ratings[name] = []
  for i, memory in enumerate(memories[name]):
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this.".format(name, plans[name], locations[name], str(global_time), memory)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    memory_ratings[name].append((res, rating))
  print(memory_ratings[name])

[('4 (important): I am very committed to achieving my goal today.', 4), ("3/10. I care enough to keep checking the web for updates on Daran Edermath's progress and to try and make it a point to talk to him in person. However, the time-related stuff gets upsetting. I'm trying to focus on my goal tonight and I cant afford any distractions.", 3), ('4/5. I care very much about Linne and her accomplishment of the same goal I am striving for.', 4), ('1/5. The memory of Halia Thornton has not been addressed.', 1), ('5. This is very important to me since I am trying to reach my goal of starting a new business in Phandalin.', 5), ('5 stars. The importance of this memory for Toblen cannot be overstated, as it is a crucial part of his journey in achieving his goal of starting a new business in the town.', 5), ('4 I care greatly about this, as this is important for the development of the town.', 4), ('Rating: 2/5. I care deeply about what Terrill and Conrad are doing in preparation for their upcom

# Compress Memories

In [12]:
MEMORY_LIMIT = 10
compressed_memories = {}
for name in town_people.keys():
  memories_sorted = sorted(
        memory_ratings[name],
        key=lambda x: x[1]
    )[::-1]
  relevant_memories = memories_sorted[:MEMORY_LIMIT]
  # print(name, relevant_memories)
  memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
  prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
  res = generate(prompt_meta.format(prompt))
  compressed_memories[name] = '[Recollection at Time {}:00: {}]'.format(str(global_time), res)
  compressed_memories_all[name].append(compressed_memories[name])

In [13]:
place_ratings = {}

for name in town_people.keys():
  place_ratings[name] = []
  for area in town_areas.keys():
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
    res = generate(prompt_meta.format(prompt))
    rating = get_rating(res)
    max_attempts = 2
    current_attempt = 0
    while rating is None and current_attempt<max_attempts:
      rating = get_rating(res)
      current_attempt += 1
    if rating is None:
      rating = 0
    place_ratings[name].append((area, rating, res))
  place_ratings_sorted = sorted(
      place_ratings[name],
      key=lambda x: x[1]
  )[::-1]
  if place_ratings_sorted[0][0] != locations[name]:
    new_recollection = '[Recollection at Time {}:00: {}]'.format(str(global_time), 'I then moved to {}.'.format(place_ratings_sorted[0][0]))
    compressed_memories_all[name].append(new_recollection)
  locations[name] = place_ratings_sorted[0][0]


# Put it all together

In [14]:
for repeats in range(5):
  global_time += 1
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)

    for name in people:
      prompt = "You are {}. Your plans are: {}. You are currently in {} with the following description: {}. Your memories are: {}. It is currently {}:00. The following people are in this area: {}. You can interact with them.".format(name, plans[name], location, town_areas[location], '\n'.join(compressed_memories_all[name][-5:]), str(global_time), ', '.join(people))
      people_description = []
      for i in people:
        people_description.append(i+': '+town_people[i])
      prompt += ' You know the following about people: ' + '. '.join(people_description)
      memory_text = '. '.join(memories[name][-10:])
      prompt += "What do you do in the next hour? Use at most 10 words to explain."
      action_prompts[name] = prompt
  action_results = {}
  for name in town_people.keys():
    action_results[name] = generate(prompt_meta.format(action_prompts[name]))
    # Now clean the action
    prompt = """
    Convert the following paragraph to first person past tense:
    "{}"
    """.format(action_results[name])
    action_results[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print(name, locations[name], global_time, action_results[name])
  action_emojis = {}
  for name in town_people.keys():
    prompt = """
    Convert the following paragraph to a tuple (Action, Object):
    "{}"
    """.format(action_results[name])
    action_emojis[name] = generate(prompt_meta.format(prompt)).replace('"', '').replace("'", '')
    print('    - Emoji Representation:', name, locations[name], global_time, action_emojis[name])
  action_prompts = {}
  for location in town_areas.keys():
    people = []
    for i in town_people.keys():
      if locations[i] == location:
        people.append(i)

    for name in people:
      for name_two in people:
        memories[name].append('[Time: {}. Person: {}. Memory: {}]\n'.format(str(global_time), name_two, action_results[name_two]))

  memory_ratings = {}
  for name in town_people.keys():
    memory_ratings[name] = []
    for i, memory in enumerate(memories[name]):
      prompt = "You are {}. Your plans are: {}. Your memories are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Give a rating, between 1 and 5, to how much you care about this.".format(name, plans[name], '\n'.join(compressed_memories_all[name][-5:]), locations[name], str(global_time), memory)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      memory_ratings[name].append((res, rating))

  compressed_memories = {}
  for name in town_people.keys():
    memories_sorted = sorted(
          memory_ratings[name],
          key=lambda x: x[1]
      )[::-1]
    relevant_memories = memories_sorted[:MEMORY_LIMIT]
    memory_string_to_compress = '.'.join([a[0] for a in relevant_memories])
    prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You observe the following: {}. Summarize these memories in one sentence.".format(name, plans[name], locations[name], str(global_time), memory_string_to_compress)
    res = generate(prompt_meta.format(prompt))
    compressed_memories[name] = '[Recollection at Time {}:00: {}]'.format(str(global_time), res)
    compressed_memories_all[name].append(compressed_memories[name])

  place_ratings = {}

  for name in town_people.keys():
    place_ratings[name] = []
    for area in town_areas.keys():
      prompt = "You are {}. Your plans are: {}. You are currently in {}. It is currently {}:00. You have the following memories: {}. Give a rating, between 1 and 5, to how likely you are likely to be at {} the next hour.".format(name, plans[name], locations[name], str(global_time), compressed_memories[name], area)
      res = generate(prompt_meta.format(prompt))
      rating = get_rating(res)
      max_attempts = 2
      current_attempt = 0
      while rating is None and current_attempt<max_attempts:
        rating = get_rating(res)
        current_attempt += 1
      if rating is None:
        rating = 0
      place_ratings[name].append((area, rating, res))
    place_ratings_sorted = sorted(
        place_ratings[name],
        key=lambda x: x[1] )[::-1]
    if place_ratings_sorted[0][0] != locations[name]:
      new_recollection = '[Recollection at Time {}:00: {}]'.format(str(global_time), 'I then moved to {}.'.format(place_ratings_sorted[0][0]))
      compressed_memories_all[name].append(new_recollection)
    locations[name] = place_ratings_sorted[0][0]


Toblen Stonehill Alderleaf Farm 9 I made contact with Toblen Stonehill and enquired about the current status of the town and nearby buildings. I also investigated the possibility of hosting a meeting with Toblen Stonehill to discuss potential opportunities for starting a new business in Phandalin.
Daran Edermath Tresendar Manor 9 Daran Edermath explored Tresendar Manor and met Terrill Bloodscar.
Linene Graywind Shrine of Luck 9 Linene entered the shrine of luck with a hope of making some sales.
Halia Thornton Townmaster’s Hall 9 I engaged in conversations with Halia, Qelline, and Conrad. I planned to meet the innkeeper and arrange a meeting between myself, the town’s leaders, and the townsfolk. We discussed strategies to increase the safety of the area, such as arming the miners or luring members of the Redbrand. We had a long, hard struggle, but eventually, the Redbrand were defeated.
Qelline Alderleaf Townmaster’s Hall 9 I have an hour until town meeting starts. I greet Halia and tal